In [2]:
import torch
from torchvision import transforms
from facenet_pytorch import InceptionResnetV1
from PIL import Image
import numpy as np
import os
from sklearn.metrics import pairwise_distances, normalized_mutual_info_score, f1_score
from sklearn.svm import OneClassSVM
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from collections import Counter
from scipy.stats import mode
import cupy as cp
import numpy as np
from sklearn.cluster import AgglomerativeClustering
import subprocess

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")

model = InceptionResnetV1(pretrained='casia-webface').eval().to(device)

lfw_dir = 'lfw-deepfunneled' 

preprocess = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor()
])

batch_size = 32 
embeddings = []
labels = []

img_data = []
label_data = []

for person_name in os.listdir(lfw_dir):
    person_dir = os.path.join(lfw_dir, person_name)
    if os.path.isdir(person_dir):
        for img_name in os.listdir(person_dir):
            img_path = os.path.join(person_dir, img_name)
            img = Image.open(img_path).convert('RGB')
            img_data.append(preprocess(img))
            label_data.append(person_name)

            # Process in batches
            if len(img_data) == batch_size:
                batch = torch.stack(img_data).to(device) 
                with torch.no_grad():
                    batch_embeddings = model(batch)
                embeddings.extend(batch_embeddings.cpu().numpy())
                labels.extend(label_data)
                img_data, label_data = [], []

if img_data:
    batch = torch.stack(img_data).to(device)
    with torch.no_grad():
        batch_embeddings = model(batch)
    embeddings.extend(batch_embeddings.cpu().numpy())
    labels.extend(label_data)

features = np.array(embeddings)
print(features.shape)
np.save("features.npy", features)
np.save("labels.npy", labels)

GPU is available
(13233, 512)


In [24]:
features = np.load("features.npy")
print(features.shape)

# Step 2.1: Compute the distance matrix
distance_matrix = pairwise_distances(features, metric='cosine')
epsilon = 0.23

# Step 2.2: Construct neighborhoods
neighborhoods = [np.where(distance_matrix[i] <= epsilon)[0] for i in range(len(features))]

# Step 2.3: Train One-Class SVM models
svdd_models = []
gamma = 1.0 / features.shape[1]  # gamma as given in LIBSVM

for i, neighbors in enumerate(tqdm(neighborhoods, desc="Training One-Class SVM Models")):
    if len(neighbors) < 1:
        svdd_models.append(None)
        continue

    svm_features = features[neighbors]
    
    model = OneClassSVM(kernel='rbf', gamma=gamma, nu=0.5)  # nu as given in LIBSVM
    model.fit(svm_features)
    
    svdd_models.append(model)

(13233, 512)


Training One-Class SVM Models: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 13233/13233 [00:47<00:00, 281.20it/s]


In [28]:
# Initialize GPU arrays
features_gpu = cp.array(features)
similarity_matrix_gpu = cp.zeros((len(features), len(features)))

# Precompute all decision values for each feature neighborhood to minimize calls
decision_values_gpu = cp.zeros((len(features), len(features)))

# Convert features batch to CPU for each SVDD model in batches
for i in tqdm(range(len(features)), desc="Batch Computing Decision Values"):
    feature_batch_cpu = cp.asnumpy(features_gpu)  # Transfer all features to CPU at once for batching
    decision_values = svdd_models[i].decision_function(feature_batch_cpu)  # Run decision function on batch
    decision_values_gpu[i, :] = cp.array(decision_values)  # Store back in GPU array

cp.save('decision_values_gpu.npy', decision_values_gpu)

Batch Computing Decision Values: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 13233/13233 [36:49<00:00,  5.99it/s]


In [52]:
decision_values_gpu = cp.load('decision_values_gpu.npy')
features_gpu = cp.array(features)

similarity_matrix_gpu = cp.zeros((len(features), len(features)))
weighted_similarity_matrix_gpu = cp.zeros((len(features), len(features)))
epsilon = 1e-5 

# Step 1: Calculate maximum neighborhood size and precompute neighborhood centers
neighborhood_lengths = [len(n) for n in neighborhoods]
max_neighborhood_size = max(neighborhood_lengths)

# Create Neighborhood Index Matrix as a NumPy Array First
neighborhood_indices_np = np.full((len(features), max_neighborhood_size), -1, dtype=np.int32)

# Fill the neighborhood indices matrix
for i, neighborhood in enumerate(neighborhoods):
    neighborhood_indices_np[i, :len(neighborhood)] = np.array(neighborhood, dtype=np.int32)

# Convert the filled NumPy array to a CuPy array
neighborhood_indices = cp.array(neighborhood_indices_np)

# Precompute centers for all neighborhoods and store them in a single matrix
centers_gpu = cp.zeros((len(features), features_gpu.shape[1]))
for i in range(len(features)):
    neighbors_i = neighborhood_indices[i, neighborhood_indices[i] >= 0]
    centers_gpu[i] = cp.mean(features_gpu[neighbors_i], axis=0)

# Step 3: Compute Similarity Matrices in Smaller Batches with Sub-batching
batch_size = 500  
sub_batch_size = 100

total_pairs = len(features) * (len(features) + 1) // 2
with tqdm(total=total_pairs, desc="Computing Pairwise Similarities") as pbar:
    for i in range(len(features)):
        neighbors_i = neighborhood_indices[i, neighborhood_indices[i] >= 0]
        center_i = centers_gpu[i]

        for j_start in range(i, len(features), batch_size):
            j_end = min(j_start + batch_size, len(features))

            for j_sub_start in range(j_start, j_end, sub_batch_size):
                j_sub_end = min(j_sub_start + sub_batch_size, j_end)

                neighbors_j_indices_sub = neighborhood_indices[j_sub_start:j_sub_end]
                centers_j_sub = centers_gpu[j_sub_start:j_sub_end]

                distances_ij_sub = cp.linalg.norm(features_gpu[neighbors_j_indices_sub] - center_i, axis=2) + epsilon
                weights_ij_sub = 1 / distances_ij_sub

                distances_ji_sub = cp.linalg.norm(features_gpu[neighbors_i] - centers_j_sub[:, None, :], axis=2) + epsilon
                weights_ji_sub = 1 / distances_ji_sub

                avg_decision_ij_sub = cp.mean(decision_values_gpu[i, neighbors_j_indices_sub], axis=1)  # Unweighted average
                avg_decision_ji_sub = cp.mean(decision_values_gpu[j_sub_start:j_sub_end, neighbors_i], axis=1)  # Unweighted average

                weighted_avg_decision_ij_sub = cp.mean(decision_values_gpu[i, neighbors_j_indices_sub] * weights_ij_sub, axis=1)  # Weighted average
                weighted_avg_decision_ji_sub = cp.mean(decision_values_gpu[j_sub_start:j_sub_end, neighbors_i] * weights_ji_sub, axis=1)  # Weighted average

                similarity_matrix_gpu[i, j_sub_start:j_sub_end] = similarity_matrix_gpu[j_sub_start:j_sub_end, i] = 0.5 * (avg_decision_ij_sub + avg_decision_ji_sub)  # Unweighted
                weighted_similarity_matrix_gpu[i, j_sub_start:j_sub_end] = weighted_similarity_matrix_gpu[j_sub_start:j_sub_end, i] = 0.5 * (weighted_avg_decision_ij_sub + weighted_avg_decision_ji_sub)  # Weighted

                pbar.update(j_sub_end - j_sub_start)
                
# Save both similarity matrices in GPU
cp.save("similarity_matrix.npy", similarity_matrix_gpu)
cp.save("weighted_similarity_matrix.npy", weighted_similarity_matrix_gpu)


Computing Pairwise Similarities: 100%|██████████████████████████████████████████████████████████████████████████████████| 87562761/87562761 [2:51:00<00:00, 8533.83it/s]


In [53]:
cp.save("decision_values_gpu",decision_values_gpu)
similarity_matrix_cpu = similarity_matrix_gpu.get()  # Move to CPU
weighted_similarity_matrix_cpu = weighted_similarity_matrix_gpu.get()  # Move to CPU

np.save("similarity_matrix_cpu.npy", similarity_matrix_cpu)
np.save("weighted_similarity_matrix_cpu.npy", weighted_similarity_matrix_cpu)

In [17]:
# Load True labels
labels = np.load("labels.npy")
similarity_matrix = np.load("similarity_matrix_cpu.npy")
k = 5  # Number of neighbors as per the algorithm

# Step 1: Build M1 and D matrices
def build_neighborhood_matrices(similarity_matrix, k):
    N = similarity_matrix.shape[0]
    M1 = np.zeros((N, k+1), dtype=int)
    D = np.zeros((N, k+1))
    
    for i in range(N):
        # Get indices of k nearest neighbors based on similarity or distance
        nearest_indices = np.argsort(-similarity_matrix[i])[:k+1]
        M1[i] = nearest_indices
        D[i] = similarity_matrix[i, nearest_indices]
        
    return M1, D

M1, D = build_neighborhood_matrices(similarity_matrix, k)

# Step 2: Construct M2 matrix with Mutual Neighborhood Values
def construct_M2(M1, k):
    N = M1.shape[0]
    M2 = np.full((N, k+1), 100)  # Fill with arbitrary large value

    for i in range(N):
        for j in range(1, k+1):
            neighbor = M1[i, j]
            if i in M1[neighbor, 1:k+1]:
                M2[i, j] = j  # Assign MNV based on neighborhood depth
            
    return M2

M2 = construct_M2(M1, k)

def perform_clustering(M1, D, M2):
    N= 13233;
    labels = np.arange(N);
    clusters = {i: [i] for i in range(N)}  # Initialize each point in its own cluster
    current_clusters = N  # Start with each sample as its own cluster

    # Iterate over MNV values, starting from 2 up to the maximum (2 * k)
    for mnv in range(2, 2*k + 1):
        pairs_to_merge = []
        
        # Collect all pairs with the current MNV value
        for i in range(N):
            for j in range(1, k+1):
                if M2[i, j] == mnv:
                    neighbor = M1[i, j]
                    dist = D[i, j]
                    pairs_to_merge.append((dist, i, neighbor))
        
        # Sort pairs by distance to prioritize merging closest pairs first
        pairs_to_merge.sort()

        # Merge pairs while ensuring each pair is only merged if in different clusters
        for dist, i, neighbor in pairs_to_merge:
            if labels[i] != labels[neighbor]:  # Only merge if they are in different clusters
                new_label = min(labels[i], labels[neighbor])
                old_label = max(labels[i], labels[neighbor])

                # Check if old_label is still in clusters before proceeding
                if old_label in clusters:
                    labels[labels == old_label] = new_label  # Update labels to reflect the merge
                    clusters[new_label].extend(clusters[old_label])
                    del clusters[old_label]  # Safely delete the old cluster entry
                    current_clusters -= 1  # Reduce cluster count after each merge
                
        # After each MNV level, check if clusters are naturally separated
        if current_clusters == 1:
            print("All data points merged into a single cluster.")
            break

    return labels, clusters

# Apply the clustering function without a target cluster count
final_labels, final_clusters = perform_clustering(M1, D, M2)

print("Clustering done")

Clustering done


In [12]:
ground_truth_labels = np.load("labels.npy")

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(ground_truth_labels)
mapped_clusters = np.zeros_like(encoded_labels)

for cluster_id, indices in final_clusters.items():
    cluster_indices = np.array(indices)
    most_common_label = mode(encoded_labels[cluster_indices])[0][0]
    mapped_clusters[cluster_indices] = most_common_label

In [13]:
def bcubed_fmeasure(final_clusters, encoded_labels):    
    n_samples = len(encoded_labels)
    precision_list = []
    recall_list = []

    # Iterate over each sample
    for i in range(n_samples):
        true_class = encoded_labels[i]

        for cluster_id, indices in final_clusters.items():
            if i in indices:
                predicted_cluster = cluster_id
                break

        cluster_indices = np.array(final_clusters[predicted_cluster])
        precision = np.sum(encoded_labels[cluster_indices] == true_class) / len(cluster_indices)

        class_indices = np.where(encoded_labels == true_class)[0]
        recall = np.sum(encoded_labels[cluster_indices] == true_class) / len(class_indices)

        if precision + recall == 0:
            f1 = 0  # Avoid division by zero
        else:
            f1 = 2 * precision * recall / (precision + recall)

        precision_list.append(precision)
        recall_list.append(recall)

    bcubed_f1 = np.mean(precision_list)
    return bcubed_f1

In [14]:
f_measure = f1_score(encoded_labels, mapped_clusters, average='weighted')
nmi_score = normalized_mutual_info_score(final_labels, ground_truth_labels)
b_cubed_f_measure = bcubed_fmeasure(final_clusters, ground_truth_labels)

print(f"F1-measure: {f_measure}")
print(f"NMI: {nmi_score}")
print(f"B Cubed F-measure: {b_cubed_f_measure}")

F1-measure: 0.9252946252709756
NMI: 0.8895425126519361
B Cubed F-measure: 0.9267812260306397


In [ ]:
# Trying out LIBSVM by locally downloading it (NOT WORKING)

In [ ]:
# Load your features
features = np.load("features.npy")
print(features.shape)

# Step 2.1: Compute the distance matrix
distance_matrix = pairwise_distances(features, metric='cosine')
epsilon = 0.23

# Step 2.2: Construct neighborhoods
neighborhoods = [np.where(distance_matrix[i] <= epsilon)[0] for i in range(len(features))]

# Step 2.3: Train SVDD models using the compiled LIBSVM
svdd_models = []

# Define paths to the compiled LIBSVM executables
svm_train_path = '/home/ipr_students/Divij/libsvm-3.22/svm-train'

for i, neighbors in enumerate(tqdm(neighborhoods, desc="Training SVDD Models")):
    if len(neighbors) < 1:
        svdd_models.append(None)
        continue
    
    # Get features for the neighborhood
    svm_features = features[neighbors]
    labels = [1] * len(svm_features)
    
    # Write features to a temporary training file in LIBSVM format
    train_file = 'training_data.txt'
    with open(train_file, 'w') as f:
        for label, feature in zip(labels, svm_features):
            # Create a line in LIBSVM format
            line = f"{label} " + " ".join(f"{j + 1}:{val}" for j, val in enumerate(feature, start=1))
            f.write(line + "\n")  # Ensure a newline at the end of each line

    params = f'-s 5'

    # Train the SVDD model using the LIBSVM executable
    model_file = 'model_file'
    subprocess.run([svm_train_path, params, train_file, model_file])

    # Store the model file name for reference
    svdd_models.append(model_file)

In [ ]:
# EARLIER VERSION (PHASE 1) : THIS CODE IS NOT OPTIMISED FOR GPU : CALCULATING SIMILARITY MATRICES
similarity_matrix = np.zeros((len(features), len(features)))
similarity_matrix2 = np.zeros((len(features), len(features)))
epsilon = 1e-5  # Small constant to avoid division by zero

for i in tqdm(range(len(features)), desc="Computing Similarities"):
    for j in range(i, len(features)):  # Symmetric matrix, only compute once
        # Initialize variables for the sums
        sum_decision_ij = 0
        sum_decision_ji = 0
        weighted_sum_decision_ij = 0
        weighted_sum_decision_ji = 0
        # Compute sums for neighbors of j in i's neighborhood
        for z in neighborhoods[j]:
            decision_value_i = svdd_models[i].decision_function(features[z].reshape(1, -1))[0]
            sum_decision_ij += decision_value_i
            # Calculate and add weighted decision function
            weight_i = 1 / (np.linalg.norm(features[z] - features[i]) + epsilon)
            weighted_sum_decision_ij += weight_i * decision_value_i

        # Compute sums for neighbors of i in j's neighborhood
        for z in neighborhoods[i]:
            decision_value_j = svdd_models[j].decision_function(features[z].reshape(1, -1))[0]
            sum_decision_ji += decision_value_j
            
            # Calculate and add weighted decision function
            weight_j = 1 / (np.linalg.norm(features[z] - features[j]) + epsilon)
            weighted_sum_decision_ji += weight_j * decision_value_j

        # Calculate average decision values
        if len(neighborhoods[j]) > 0:
            avg_decision_ij = sum_decision_ij / len(neighborhoods[j])
            avg_weighted_decision_ij = weighted_sum_decision_ij / len(neighborhoods[j])
        else:
            avg_decision_ij = 0
            avg_weighted_decision_ij = 0

        if len(neighborhoods[i]) > 0:
            avg_decision_ji = sum_decision_ji / len(neighborhoods[i])
            avg_weighted_decision_ji = weighted_sum_decision_ji / len(neighborhoods[i])
        else:
            avg_decision_ji = 0
            avg_weighted_decision_ji = 0
        # Calculate similarity and weighted similarity
        similarity_matrix[i, j] = similarity_matrix[j, i] = 0.5 * (avg_decision_ij + avg_decision_ji)
        similarity_matrix2[i, j] = similarity_matrix2[j, i] = 0.5 * (avg_weighted_decision_ij + avg_weighted_decision_ji)

np.save("similarity_matrix.npy", similarity_matrix)
np.save("similarity_matrix2.npy", similarity_matrix2)